In [0]:
#در این قسمت ابتدا کتابخانه های مورد نیاز را نصب میکنیم
!pip install emoji
!pip install --upgrade tensorflow==2.0.0
!pip install --upgrade tensorflow-gpu==2.0.0
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade

In [0]:
# در این قسمت کتابخانه های لازم را ایمپورت میکنیم
import pickle as pkl
import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt

from __future__ import unicode_literals
from hazm import *


import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
print(tf.__version__)

2.0.0


# section 1

In [0]:
# آدرس فایل جملات
sentence_data = 'drive/My Drive/Untitled folder/fa_2.xlsx'

# خواندن فایل ورودی و جدا کردن جملات و لیبل آن ها به صورت دو لیست متفاوت
data = pd.read_excel(sentence_data)
sentences = list(data.text)
labels = list(data.label)

# ذخیره جملات هر کلاس در یک لیست جداگانه
neg_sentences = []
pos_sentences = []
med_sentences = []

for sen, l in zip(sentences, labels):
  if l == 'neg':
    neg_sentences.append(sen)
  if l == 'pos':
    pos_sentences.append(sen)
  if l == 'med':
    med_sentences.append(sen)

print("We have %d neg sentences"%len(neg_sentences))
print("We have %d pos sentences"%len(pos_sentences))
print("We have %d med sentences"%len(med_sentences))

We have 3070 neg sentences
We have 1619 pos sentences
We have 311 med sentences


In [0]:
# نوشتن یک نرمالایزر برای نرمال کردن دیتا با استفاده از هضم 
def Normalizer_text(input_text, normalizer):

  # حذف فاصله های اضافی در متن
  input_text = input_text.rstrip('\r\n').strip()
  #حذف نام کاربری از متن
  normalized_text = re.sub('@[^\s]+','', input_text)
  # حذف آدرس های اینترنتی از متن
  normalized_text = re.sub(r"http\S+", "", normalized_text)
  # نرمال کردن متن با هضم
  normalized_text = normalizer.normalize(normalized_text)
  # حذف برخی از علامت های نگارشی
  normalized_text = normalized_text.replace('«', ' ').replace('»', ' ')\
    .replace('"', ' ').replace('#', ' ').replace('-', ' ').replace('_', ' ')\
    .replace('*', ' ').replace('…', ' ').replace("'", ' ').replace('\n\n', ' ')\
    .replace('\n', ' ').replace('^', ' ')
  # توکن کردن دیتا
  tokenized_text = word_tokenize(normalized_text)
  # حذف حروفی که بیشتر از دوبار پشت سر هم تکرار شده اند.
  token_list = [re.sub(r'(.)\1+', r'\1\1', token) for token in tokenized_text]

  return token_list

In [0]:
normalizer = Normalizer()

X_train=[]
y_train=[]

X_test=[]
y_test=[]

# تقسیم دیتا به دو بخش آموزشی و تست
for sen in neg_sentences[:2870]:
  X_train.append(Normalizer_text(sen, normalizer))
  y_train.append('neg')

for sen in neg_sentences[2870:]:
  X_test.append(Normalizer_text(sen, normalizer))
  y_test.append('neg')

for sen in pos_sentences[:1540]:
  X_train.append(Normalizer_text(sen, normalizer))
  y_train.append('pos')

for sen in pos_sentences[1540:]:
  X_test.append(Normalizer_text(sen, normalizer))
  y_test.append('pos')

for sen in med_sentences[:280]:
  X_train.append(Normalizer_text(sen, normalizer))
  y_train.append('med')

for sen in med_sentences[280:]:
  X_test.append(Normalizer_text(sen, normalizer))
  y_test.append('med')

In [0]:
# این تابع برای تبدیل کلمات جملات به ایندکس نوشته شده است
def make_numberized_samples():
  my_tokens=[]
  for sen in X_train:
    for token in sen:
      my_tokens.append(token)

  print('We have {} tokens.'.format(len(my_tokens)))
  my_tokens = set(my_tokens)
  print('We have {} unique tokens.'.format(len(my_tokens)))

  word2idx = {}
  idx2word = {}
  word2idx['UNK'] = 1
  idx2word[1] = 'UNK'
  for i, token in enumerate(my_tokens):
    word2idx[token] = i+2
    idx2word[i+2] = token

  label2idx = {}
  idx2label = {}
  label2idx['neg'] = 0
  label2idx['pos'] = 1
  label2idx['med'] = 2
  idx2label[0] = 'neg'
  idx2label[1] = 'pos'
  idx2label[2] = 'med'

  numberized_sen_train=[]
  numberized_labels_train=[]

  numberized_sen_test=[]
  numberized_labels_test=[]

  for sen in X_train:
    x=[]
    for t in sen:
      x.append(word2idx[t])
    numberized_sen_train.append(x)

  for l in y_train:
    numberized_labels_train.append(label2idx[l])

  for sen in X_test:
    x=[]
    for t in sen:
      try:
        x.append(word2idx[t])
      except:
        x.append(1)
    numberized_sen_test.append(x)

  for l in y_test:
    numberized_labels_test.append(label2idx[l])
  
  numberized_sen_train=np.asarray(numberized_sen_train)
  numberized_labels_train=np.asarray(numberized_labels_train)
  numberized_sen_test=np.asarray(numberized_sen_test)
  numberized_labels_test=np.asarray(numberized_labels_test)

  indices = np.arange(numberized_sen_train.shape[0])
  np.random.shuffle(indices)
  np.random.shuffle(indices)
  numberized_sen_train = numberized_sen_train[indices]
  numberized_labels_train = numberized_labels_train[indices]

  indices = np.arange(numberized_sen_test.shape[0])
  np.random.shuffle(indices)
  np.random.shuffle(indices)
  numberized_sen_test = numberized_sen_test[indices]
  numberized_labels_test = numberized_labels_test[indices]


  pickle_data = [numberized_sen_train, numberized_labels_train,\
                numberized_sen_test, numberized_labels_test,\
                    word2idx, idx2word, label2idx, idx2label]

  pickle_address = 'drive/My Drive/Untitled folder/data_11.pkl'
  with open(pickle_address, 'wb') as f:
    pkl.dump(pickle_data, f)

  print('Saved as pickle file')

make_numberized_samples()

We have 123114 tokens.
We have 20812 unique tokens.
Saved as pickle file


In [0]:
# فایل ذخیره شده در تابع قبل را میخوانیم
pickle_address = 'drive/My Drive/Untitled folder/data_1.pkl'
with open(pickle_address, 'rb') as myData:
  numberized_sen_train, numberized_labels_train,\
    numberized_sen_test, numberized_labels_test,\
      word2idx, idx2word, label2idx, idx2label = pkl.load(myData)
max_len = max(len(sentences) for sentences in numberized_sen_train)

In [0]:
# جملات ورودی در شبکه باید طول یکسانی داشته باشند. با این تابع طول همه جملات را یکسان میکنیم
numberized_sen_train = pad_sequences(numberized_sen_train, maxlen = max_len, padding='post')
numberized_sen_test = pad_sequences(numberized_sen_test, maxlen = max_len, padding='post')

In [0]:
EMBEDDING_DIM = 300
n_epochs = 10
BATCH_SIZE = 64

In [0]:
# تعریف لایه امبدینگ برای تبدیل کلمات به بردار 300 بعدی
embedding_layer = Embedding(len(word2idx)+1,\
                            EMBEDDING_DIM,\
                            input_length=max_len,\
                            trainable=True)

# تعریف لایه ورودی 
sequence_input = Input(shape=(max_len, ), dtype=tf.int32)

# ورودی را با استفاده از لایه امبدینگ به بردار تبدیل میکنیم
embedded_sequence = embedding_layer(sequence_input)

# تعریف لایه RNN
# تابع فعالیت relu - تعداد نورون 128
bilstm_1 = Bidirectional(SimpleRNN(units=128, activation='relu',\
                              return_sequences=True))(embedded_sequence)
# استفاده از dropout ->> جلوگیری از بیش براز
bilstm_1 = Dropout(0.3)(bilstm_1)

# تعریف لایه RNN
# تابع فعالیت relu - تعداد نورون 128
bilstm_2 = Bidirectional(SimpleRNN(units=128, activation='relu',\
                              return_sequences=False))(bilstm_1)                              
# استفاده از dropout ->> جلوگیری از بیش براز
bilstm_2 = Dropout(0.3)(bilstm_2)

# تعریف لایه پرسپترون برای استخراج ویژگی های بهتر
dense_1 = Dense(units=64, activation='relu')(bilstm_2)
# استفاده از dropout ->> جلوگیری از بیش براز
dense_1 = Dropout(0.3)(dense_1)
# لایه خروجی با تابع فعالیت softmax
pred = Dense(units=len(label2idx), activation='softmax')(dense_1)

# تعریف ورودی و مشخص کردن ورودی و خروجی
model = Model(inputs=[sequence_input], outputs=pred)
# کامپایل کردن مدل و تعریف تابع خطا و بهینه ساز
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics= ['accuracy'])

# گرفتن سامری از مدل
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 99)]              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 99, 300)           6244200   
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 99, 256)           109824    
_________________________________________________________________
dropout_24 (Dropout)         (None, 99, 256)           0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 256)               98560     
_________________________________________________________________
dropout_25 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                1644

In [0]:
# شروع آموزش
history = model.fit(numberized_sen_train, numberized_labels_train,\
                    validation_data=(numberized_sen_test, numberized_labels_test),\
                     epochs=n_epochs, batch_size=BATCH_SIZE, verbose=1)

In [0]:
# ذخیره مدل آموزش دیده شده 
model.save('drive/My Drive/Untitled folder/1.h5')

In [0]:
# لود کردن مدل
from tensorflow.keras.models import load_model
model = load_model('drive/My Drive/Untitled folder/1.h5')

In [0]:
# ارزیابی مدل
loss, acc = model.evaluate(numberized_sen_test, numberized_labels_test)
print('loss: {}, accuracy: {}'.format(loss, acc))

310/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

# section 2

In [0]:
# آدرس فایل جملات
sentence_data = 'drive/My Drive/Untitled folder/fa_2.xlsx'

# خواندن فایل ورودی و جدا کردن جملات و لیبل آن ها به صورت دو لیست متفاوت
data = pd.read_excel(sentence_data)
sentences = list(data.text)
labels = list(data.label)

# ذخیره جملات هر کلاس در یک لیست جداگانه
neg_sentences = []
pos_sentences = []
med_sentences = []

for sen, l in zip(sentences, labels):
  if l == 'neg':
    neg_sentences.append(sen)
  if l == 'pos':
    pos_sentences.append(sen)
  if l == 'med':
    med_sentences.append(sen)

print("We have %d neg sentences"%len(neg_sentences))
print("We have %d pos sentences"%len(pos_sentences))
print("We have %d med sentences"%len(med_sentences))

We have 3070 neg sentences
We have 1619 pos sentences
We have 311 med sentences


In [0]:
# نوشتن یک نرمالایزر برای نرمال کردن دیتا با استفاده از هضم 
def Normalizer_text(input_text, normalizer):

  # حذف فاصله های اضافی در متن
  input_text = input_text.rstrip('\r\n').strip()
  #حذف نام کاربری از متن
  normalized_text = re.sub('@[^\s]+','', input_text)
  # حذف آدرس های اینترنتی از متن
  normalized_text = re.sub(r"http\S+", "", normalized_text)
  # نرمال کردن متن با هضم
  normalized_text = normalizer.normalize(normalized_text)
  # حذف برخی از علامت های نگارشی
  normalized_text = normalized_text.replace('«', ' ').replace('»', ' ')\
    .replace('"', ' ').replace('#', ' ').replace('-', ' ').replace('_', ' ')\
    .replace('*', ' ').replace('…', ' ').replace("'", ' ').replace('\n\n', ' ')\
    .replace('\n', ' ').replace('^', ' ')
  # توکن کردن دیتا
  tokenized_text = word_tokenize(normalized_text)
  # حذف حروفی که بیشتر از دوبار پشت سر هم تکرار شده اند.
  token_list = [re.sub(r'(.)\1+', r'\1\1', token) for token in tokenized_text]

  return token_list

In [0]:
normalizer = Normalizer()

X_train=[]
y_train=[]

X_test=[]
y_test=[]

# تقسیم دیتا به دو بخش آموزشی و تست
for sen in neg_sentences[:2870]:
  X_train.append(Normalizer_text(sen, normalizer))
  y_train.append('neg')

for sen in neg_sentences[2870:]:
  X_test.append(Normalizer_text(sen, normalizer))
  y_test.append('neg')

for sen in pos_sentences[:1540]:
  X_train.append(Normalizer_text(sen, normalizer))
  y_train.append('pos')

for sen in pos_sentences[1540:]:
  X_test.append(Normalizer_text(sen, normalizer))
  y_test.append('pos')

for sen in med_sentences[:280]:
  X_train.append(Normalizer_text(sen, normalizer))
  y_train.append('med')

for sen in med_sentences[280:]:
  X_test.append(Normalizer_text(sen, normalizer))
  y_test.append('med')

In [0]:
# خواندن فایل ویژگی های هر کلمه
word_data = 'drive/My Drive/Untitled folder/Phrases.xlsx'

# خواندن فایل و ذخیره کلمات و ویژگی ها به صورت لیست
data = pd.read_excel(word_data)
words = list(data.PersianTranslation)
pos = list(data.Positive)
neg = list(data.Negative)
anger = list(data.Anger)
anticipation = list(data.Anticipation)
disgust = list(data.Disgust)
fear = list(data.Fear)
joy = list(data.Joy)
sadness = list(data.Sadness)
surprise = list(data.Surprise)
trust = list(data.Trust)

# تعریف یک دیکشنری و ذخیره هر کلمه به همراه ویژگی آن در دیکشنری
words_features = {}

for w, p, n, ang, ant, d, f, j, sad, sur, t in zip(words, pos, neg, anger, anticipation, disgust, fear, joy, sadness, surprise, trust):
  words_features[w] = [p, n, ang, ant, d, f, j, sad, sur, t]

In [0]:
# تبدیل کلمات به بردار عددی در این تابع انجام میشود
def make_numberized_samples():
  my_tokens=[]
  for sen in X_train:
    for token in sen:
      my_tokens.append(token)

  print('We have {} tokens.'.format(len(my_tokens)))
  my_tokens = set(my_tokens)
  print('We have {} unique tokens.'.format(len(my_tokens)))

  word2idx = {}
  idx2word = {}
  word2idx['UNK'] = 1
  idx2word[1] = 'UNK'
  for i, token in enumerate(my_tokens):
    word2idx[token] = i+2
    idx2word[i+2] = token

  label2idx = {}
  idx2label = {}
  label2idx['neg'] = 0
  label2idx['pos'] = 1
  label2idx['med'] = 2
  idx2label[0] = 'neg'
  idx2label[1] = 'pos'
  idx2label[2] = 'med'

  numberized_sen_train=[]
  numberized_features_train = []
  numberized_labels_train=[]

  numberized_sen_test=[]
  numberized_features_test = []
  numberized_labels_test=[]

  for sen in X_train:
    x=[]
    for t in sen:
      x.append(word2idx[t])
    numberized_sen_train.append(x)

  for sen in X_train:
    f=[]
    for t in sen:
      try:
        f.append(np.asarray(words_features[t]))
      except:
        f.append(np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    numberized_features_train.append(f)

  for l in y_train:
    numberized_labels_train.append(label2idx[l])

  for sen in X_test:
    x=[]
    for t in sen:
      try:
        x.append(word2idx[t])
      except:
        x.append(1)
    numberized_sen_test.append(x)

  for sen in X_test:
    f=[]
    for t in sen:
      try:
        f.append(np.asarray(words_features[t]))
      except:
        f.append(np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    numberized_features_test.append(f)

  for l in y_test:
    numberized_labels_test.append(label2idx[l])
  
  numberized_sen_train=np.asarray(numberized_sen_train)
  numberized_features_train = np.asarray(numberized_features_train)
  numberized_labels_train=np.asarray(numberized_labels_train)
  numberized_sen_test=np.asarray(numberized_sen_test)
  numberized_features_test = np.asarray(numberized_features_test)
  numberized_labels_test=np.asarray(numberized_labels_test)

  indices = np.arange(numberized_sen_train.shape[0])
  np.random.shuffle(indices)
  np.random.shuffle(indices)
  numberized_sen_train = numberized_sen_train[indices]
  numberized_features_train = numberized_features_train[indices]
  numberized_labels_train = numberized_labels_train[indices]

  indices = np.arange(numberized_sen_test.shape[0])
  np.random.shuffle(indices)
  np.random.shuffle(indices)
  numberized_sen_test = numberized_sen_test[indices]
  numberized_features_test = numberized_features_test[indices]
  numberized_labels_test = numberized_labels_test[indices]


  pickle_data = [numberized_sen_train, numberized_features_train, numberized_labels_train,\
                 numberized_sen_test, numberized_features_test, numberized_labels_test,\
                    word2idx, idx2word, label2idx, idx2label]

  pickle_address = 'drive/My Drive/Untitled folder/data_2.pkl'
  with open(pickle_address, 'wb') as f:
    pkl.dump(pickle_data, f)

  print('Saved as pickle file')

make_numberized_samples()

We have 123114 tokens.
We have 20812 unique tokens.
Saved as pickle file


In [0]:
# خواندن خروجی های ذخیره شده از تابع قبل
pickle_address = 'drive/My Drive/Untitled folder/data_2.pkl'
with open(pickle_address, 'rb') as myData:
  numberized_sen_train, numberized_features_train, numberized_labels_train,\
    numberized_sen_test, numberized_features_test, numberized_labels_test,\
      word2idx, idx2word, label2idx, idx2label = pkl.load(myData)
max_len = max(len(sentences) for sentences in numberized_sen_train)

In [0]:
# جملات ورودی در شبکه باید طول یکسانی داشته باشند. با این تابع طول همه جملات را یکسان میکنیم
numberized_sen_train = pad_sequences(numberized_sen_train, maxlen = max_len, padding='post')
numberized_features_train = pad_sequences(numberized_features_train, maxlen = max_len, padding='post')

numberized_sen_test = pad_sequences(numberized_sen_test, maxlen = max_len, padding='post')
numberized_features_test = pad_sequences(numberized_features_test, maxlen = max_len, padding='post')

In [0]:
EMBEDDING_DIM = 300
n_epochs = 10
BATCH_SIZE = 128

In [0]:
# تعریف لایه امبدینگ برای تبدیل کلمات به بردار 300 بعدی
embedding_layer = Embedding(len(word2idx)+1,\
                            EMBEDDING_DIM,\
                            input_length=max_len,\
                            trainable=True)

# تعریف لایه های  ورودی 

# ورودی خود کلمات
sequence_input = Input(shape=(max_len, ), dtype=tf.int32)

# ورودی ویژگی های اضافه شده
feature_input = Input(shape=(max_len, 10), dtype=tf.float32)

# تبدیل کلمات به بردار ویژگی توسط لایه امبدینگ
embedded_sequence = embedding_layer(sequence_input)

# ترکیب بردار کلمات بدست آمده از امبدینگ و بردار ویژگی هر کلمه
concat = concatenate([embedded_sequence, feature_input], axis=2)

# تعریف لایه RNN
# تابع فعالیت relu - تعداد نورون 128
bilstm_1 = Bidirectional(SimpleRNN(units=128, activation='relu',\
                              return_sequences=True))(concat)

# استفاده از dropout ->> جلوگیری از بیش براز                              
bilstm_1 = Dropout(0.35)(bilstm_1)

# تعریف لایه RNN
# تابع فعالیت relu - تعداد نورون 128
bilstm_2 = Bidirectional(SimpleRNN(units=128, activation='relu',\
                              return_sequences=False))(bilstm_1)                              

# استفاده از dropout ->> جلوگیری از بیش براز                                 
bilstm_2 = Dropout(0.35)(bilstm_2)

# استفاده از یک لایه پرسپترون برای استخراج ویژگی مناسب تر
dense_1 = Dense(units=64, activation='relu')(bilstm_2)
# استفاده از dropout ->> جلوگیری از بیش براز   
dense_1 = Dropout(0.35)(dense_1)

# لایه خروجی با تابع فعالیت softmax
pred = Dense(units=len(label2idx), activation='softmax')(dense_1)

# تعریف ورودی و مشخص کردن ورودی و خروجی
model = Model(inputs=[sequence_input, feature_input], outputs=pred)

# کامپایل کردن مدل و تعریف تابع خطا و بهینه ساز
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics= ['accuracy'])

model.summary()

(None, 99, 10)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 99)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 99, 300)      6244200     input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 99, 10)]     0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 99, 310)      0           embedding_2[0][0]                
                                                                 input_6[0][0

In [0]:
history = model.fit([numberized_sen_train, numberized_features_train], numberized_labels_train,\
                      validation_data=([numberized_sen_test, numberized_features_test], numberized_labels_test),\
                        epochs=n_epochs, batch_size=BATCH_SIZE, verbose=1)

Train on 4690 samples, validate on 310 samples
Epoch 1/10
4690/4690 [==============================] - 44s 9ms/sample - loss: 0.8605 - accuracy: 0.6006 - val_loss: 0.8657 - val_accuracy: 0.6516
Epoch 2/10
4690/4690 [==============================] - 39s 8ms/sample - loss: 0.6846 - accuracy: 0.7139 - val_loss: 0.6801 - val_accuracy: 0.7516
Epoch 3/10
4690/4690 [==============================] - 39s 8ms/sample - loss: 0.3200 - accuracy: 0.8906 - val_loss: 0.7715 - val_accuracy: 0.7452
Epoch 4/10
4690/4690 [==============================] - 39s 8ms/sample - loss: 0.1697 - accuracy: 0.9356 - val_loss: 0.9240 - val_accuracy: 0.7710
Epoch 5/10
4690/4690 [==============================] - 39s 8ms/sample - loss: 0.1031 - accuracy: 0.9676 - val_loss: 1.3531 - val_accuracy: 0.6903
Epoch 6/10
4690/4690 [==============================] - 39s 8ms/sample - loss: 0.0727 - accuracy: 0.9800 - val_loss: 1.4107 - val_accuracy: 0.6645
Epoch 7/10
4690/4690 [==============================] - 39s 8ms/sample 

In [0]:
# ذخیره مدل
model.save('drive/My Drive/Untitled folder/2.h5')

In [0]:
# لود کردن مدل
from tensorflow.keras.models import load_model
model = load_model('drive/My Drive/Untitled folder/2.h5')

In [0]:
# ارزیابی مدل
loss, acc = model.evaluate([numberized_sen_test, numberized_features_test], numberized_labels_test)
print('loss: {}, accuracy: {}'.format(loss, acc))

310/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================